In [1]:
import wmi
import gc
from win32api import GetFileVersionInfo, HIWORD, LOWORD
wmic = wmi.WMI()
process = wmic.Win32_Process(name="cloudmusic.exe")
process = [p for p in process if '--type=' not in p.ole_object.CommandLine]
if not process:
    raise RuntimeError('No candidate process found')
elif len(process) != 1:
    raise RuntimeError('Multiple candidate processes found!')
else:
    process = process[0]
    pid = process.ole_object.ProcessId
    ver_info = GetFileVersionInfo(process.ole_object.ExecutablePath, '\\')
    ver = f"{HIWORD(ver_info['FileVersionMS'])}.{LOWORD(ver_info['FileVersionMS'])}." \
          f"{HIWORD(ver_info['FileVersionLS'])}.{LOWORD(ver_info['FileVersionLS'])}"
    print(f'found version: {ver}')
    del process
    del wmic
    gc.collect()

found version: 2.10.12.5241


In [2]:
from pyMeow import *

process = open_process(pid)
base_exe_address = get_module(process, 'cloudmusic.exe')['base']
base_dll_address = get_module(process, 'cloudmusic.dll')['base']
print('base_exe_address: 0x%X' % base_exe_address)
print('base_dll_address: 0x%X' % base_dll_address)

base_exe_address: 0x4A0000
base_dll_address: 0x797E0000


In [6]:
# Scanning for V2.x clients
import math

found_songarray = False
found_length = False
found_current = False
for offset in range(0xA7A580, 0xC20000): # Change to estimated memory range
    if found_songarray and found_length and found_current:
        break
    if not found_songarray:
        try:
            songid_array = r_uint(process, base_dll_address + offset)
            string = '1458767558'  # Change to current song ID
            song_id = r_bytes(process, songid_array, len(string) * 2).decode('utf-16')
            if song_id == string:
                print('found song array offset at 0x%X: %s' % (offset, song_id))
                found_songarray = True
        except Exception as e:
            pass

    if not found_length:
        try:
            length = r_float64(process, base_dll_address + offset)
            if math.floor(length) == 255:  # Change to song length, in seconds
                print('found length at 0x%X: %f' % (offset, length))
                found_length = True
        except Exception as e:
            pass

    if not found_current:
        try:
            current = r_float64(process, base_dll_address + offset)
            if math.floor(current) == 38:  # Change to current progress, in seconds
                print('found current at 0x%X: %f' % (offset, current))
                found_current = True
        except Exception as e:
            pass

found current at 0xA7A580: 38.851000
found song array offset at 0xB2BCB0: 1458767558
found length at 0xB2CA98: 255.205306


In [6]:
# Scanning for V3.0 clients (not working fully)
import math
from tqdm.auto import tqdm

found_songarray = False
found_current = False
offsets = [0x48, 0x30, 0x70, 0x0]  # need to find using cheat engine, see scanning.txt
for offset in tqdm(range(0x18e0000, 0x1930000)): # Change to range of cloudmusic.dll in cheat engine
    if found_current and found_songarray:
        break

    if not found_current:
        try:
            current = r_float64(process, base_dll_address + offset)
            if math.floor(current) == 16:  # Change to current progress, in seconds
                print('found current at 0x%X: %f' % (offset, current))
                found_current = True
        except Exception as e:
            pass
    
    if not found_songarray:
        try:
            songid_array = pointer_chain_64(process, base_dll_address + offset, offsets)
            string = '1458767558'  # Change to current song ID
            song_id = r_bytes(process, songid_array, 0x14)
            song_id = bytes([b for b in song_id if b <= 128])  # filter to ascii only
            song_id = song_id.decode('ascii').split('_')[0]
            if song_id == string:
                print('found song array offset at 0x%X: %s' % (offset, song_id))
                print('song array offsets: [' + ', '.join([hex(o) for o in offsets]) + ']')
                found_songarray = True
        except Exception as e:
            pass

  0%|          | 0/327680 [00:00<?, ?it/s]

found current at 0x18EF318: 16.527148
found song array offset at 0x192D6A0: 1458767558
song array offsets: [0x48, 0x30, 0x70, 0x0]
